# BackEnd for Raw Gauge Viewer
Contains classes and widgets needed to view the raw data from 1 or more gauges

In [1]:
%pylab notebook
import numpy as np
import sys
import pylab as pl
import pandas as pd
import ipywidgets as widgets
import pytimber
db = pytimber.LoggingDB()
# import os
# print 'my working directory is: '+(os.getcwd())

max_fill = db.getLHCFillData(None)['fillNumber']

fill_field = widgets.BoundedIntText(
    value=5979,
    min=0,
    max=max_fill,
    step=1,
    description='FillNo:',
    disabled=False
)


gauge_id_field = widgets.Text(
    value='VGPB.163.5R2.B.PR',
    placeholder='VGPB.163.5R2.B.PR',
    description='Gauge ID:',
    disabled=False
)



class VacuumGauge(object):
    
    def __init__(self, fillNumber, beamNumber = 1, verbose = False):
        self.verbose = verbose
        self.beamNumber = beamNumber
        
        self.db = pytimber.LoggingDB()
        
        self.fillNumber = fillNumber
        fill = self.db.getLHCFillData(self.fillNumber)
        self._t1   = fill['startTime']
        self._t2   = fill['endTime']
        
        self._getBeamIntensity()
        self._getBeamEnergy()
        self._getBeamBunchLengths()
  


    def _getBeamIntensity(self):
        if self.verbose: print('Downloading beam intensity..')
        
        variable='LHC.BCTFR.A6R4.B'+str(self.beamNumber)+':BEAM_INTENSITY'
        self.beamIntensity=self.db.get(variable, self._t1, self._t2)
        self.beamIntensity=self.beamIntensity[variable]
        
        if self.verbose:print('Beam intensity succesfully downloaded')
#%%
    
    def _getBeamEnergy(self):
        if self.verbose: print('Downloading beam energy..')
        
        variable='LHC.BOFSU:OFSU_ENERGY'
        self.beamEnergy=self.db.get(variable, self._t1, self._t2)
        self.beamEnergy=self.beamEnergy[variable]
        
        if self.verbose: print('Beam energy succesfully downloaded')

  
    
    def _getBeamBunchLengths(self):
        if self.verbose: print('Downloading beam bunch lengths..')
        
        variable='LHC.BQM.B'+str(self.beamNumber)+':BUNCH_LENGTH_MEAN'
        self.beamBunchLengths=self.db.get(variable, self._t1, self._t2)
        self.beamBunchLengths=self.beamBunchLengths[variable]
        
        if self.verbose: print('Beam bunch lengths succesfully downloaded')
    
    def dropEmpty(self):
        if not hasattr(self,'data'):
            raise ValueError("Data for Vacuum Gauge class Not Loaded, call getData()")
            
        count = 0
        for col in self.data.columns:
            if len(self.data.loc['timeStamp'][col]) == 0:
                self.data.drop(col, inplace=True, axis=1)
                count += 1
        if self.verbose: print("{} empty readings removed".format(count))
       
    
    def getData(self, variable='VGI%.PR'):
        
        self.query = variable
        if self.verbose: print('Downloading vacuum gauge data..')

        
        VG = self.db.get(variable, self._t1, self._t2)
    

        self.data = pd.DataFrame.from_dict(VG)

        index = self.data.index
        index_list = index.tolist()
        index_list[0] = 'timeStamp'
        index_list[1] = 'values'

        self.data.index = index_list
        
        if self.verbose: print('Vacuum gauge data succesfully downloaded')



    def plot(self, showPlot=True, fontsize = 10, plot_df=None):
            
        if plot_df is None:
            plot_df = self.data
        
        title = 'Plotting' + str(self.query) + 'for fill' + str(self.fillNumber)
        
        cols=cm.rainbow(np.linspace(0,1,10))
        
        pl.ioff()
        
        f, (ax1,ax2) = pl.subplots(2, sharex=True, gridspec_kw = {'height_ratios':[17, 5]}, figsize=(10, 6))
        f.suptitle(title)
        
        i=0
        for col in plot_df.columns:
            label = col
            timeStamp = plot_df.loc['timeStamp'][col]
            pressure = plot_df.loc['values'][col]
            ax1.plot(timeStamp,pressure,label=col,color=cols[i], marker='x', markerfacecolor=cols[i])
            i+=1
            
        ax1.set_ylabel('Pressures')
        ax1.set_yscale('log')
                     
        ax1_1 = ax1.twinx()
        ax1_1.grid(False)
        ax1_1.plot(self.beamIntensity[0], self.beamIntensity[1], label=
                   'LHC.BCTFR.A6R4.B'+str(self.beamNumber)+':BEAM_INTENSITY',linewidth=2)
        ax1_1.set_ylabel('Charges')
        ax1_1.legend(fontsize=fontsize,loc='upper right')
        box = ax1.get_position()
        ax1.set_position([box.x0, box.y0, box.width * 0.8, box.height])
        pytimber.set_xaxis_date()
        ax1.legend(loc='upper center',fontsize=fontsize,bbox_to_anchor=(1.18,1.00),
                  ncol=1, fancybox = True, shadow = True)
        
        # Make the y-axis label, ticks and tick labels match the line color.
        ax2.plot(self.beamEnergy[0], self.beamEnergy[1],label='LHC.BOFSU:OFSU_ENERGY', color='r')
        ax2.grid(True)
        ax2.set_ylabel('GeV')
        ax2.set_xlim(np.min(self.beamIntensity[0]), np.max(self.beamIntensity[0]))
        ax2.legend(fontsize=fontsize,loc='upper left')
        
        
        
        ax2_1 = ax2.twinx()
        ax2_1.grid(False)
        ax2_1.plot(self.beamBunchLengths[0], self.beamBunchLengths[1],label='LHC.BQM.B'+str(self.beamNumber)+':BUNCH_LENGTH_MEAN', color='b')
        ax2_1.set_ylabel('s')
        ax2_1.legend(fontsize=fontsize,loc='upper right')
        box = ax2.get_position()
        ax2.set_position([box.x0, box.y0, box.width * 0.8, box.height])
        
        date=str(datetime.datetime.now()).replace(' ','_')
                     
        if showPlot:
            f.show()
            
    def sliced_plot(self, slice_number=10, fontsize=10):
        if slice_number >= len(self.data.columns):
            slice_number = len(self.data.columns)-1
        
        slices = np.arange(0, len(self.data.columns), slice_number)
        
        for i, pos in enumerate(slices):
            if pos == slices[-1]:
                self.plot(plot_df = self.data.iloc[:, pos:])
            else:
                self.plot(plot_df = self.data.iloc[:, pos:pos+slice_number])

Populating the interactive namespace from numpy and matplotlib
